In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from wordcloud import WordCloud, STOPWORDS
#Display all the columns 
pd.set_option('display.max_columns',None)
# Display full length the column
pd.set_option('display.max_colwidth', -1)
import re
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from string import punctuation
from nltk.stem.porter import *
from gensim.models import Word2Vec,KeyedVectors
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from spellchecker import SpellChecker
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier
# nltk.download('stopwords')
# nltk.download('wordnet')

In [24]:
from html.parser import HTMLParser 
html = HTMLParser()

In [25]:
import os
os.chdir('C:\Data Science\Jupyter Notebook\GitHackathon\data\Iter')

In [26]:
clean_train_df = pd.read_csv('train_clean_data.csv')


In [27]:
pwd

'C:\\Data Science\\Jupyter Notebook\\GitHackathon\\data\\Iter'

In [28]:
test_df = pd.read_csv('../../data/raw/test.csv')
test_cdf = test_df.copy()

In [29]:
test_cdf['tweet'] = test_cdf['tweet'].str.lower()

In [30]:
test_cdf['tweet'] = test_cdf['tweet'].apply(lambda x: html.unescape(x))

In [31]:
test_cdf

,tweet_id,tweet
0,7506,"audience q: what prototyping tools do you use? sketchbooks/sharpie pens, photoshop, balsamic, google docs, axsure, etc. #myprototype #sxsw"
1,7992,at sxsw? send your best photos & videos to... {link} #citizen_journalism #cnn #ireport #photography #sxsw #cyber #iphone
2,247,@mention and here's a pic of you winning your ipad! #unsix #sxsw cc @mention @mention {link} (cont) {link}
3,7688,google marissa mayer: mobile phone as a cursor of physical location - new version of map fast and more real life like #sxsw
4,3294,#sxsw google maps is even cooler than i thought
...,...,...
1814,1550,@mention @mention @mention hmmm....how fast can #apple build a new store in time for #sxsw {link}
1815,1933,samsung galaxy s ii appears at fcc and team android #sxsw party {link} via @mention
1816,9052,@mention you could buy a new ipad 2 tmrw at the apple pop-up store at #sxsw: {link}
1817,4219,"wow very long queue of people at apple pop up store now, some have bought 3 ipads! #sxsw@mention room#nokiaconnects"


In [32]:
# Adding unnecessary char to stop words
stop_words = list(set(stopwords.words('english')))+list(punctuation)+['``', "'s", "...", "n't"]

In [33]:
# creating tokens
test_cdf['tweet'] = [nltk.word_tokenize(x) for x in test_cdf['tweet']]

In [34]:
# Removing stop words 
test_cdf['tweet'] = test_cdf['tweet'].apply(lambda row: [word for word in row if word not in stop_words])

#Applying the stemming

# stemmer = PorterStemmer()
# cdf['tweet_tokens'] = cdf['tweet_tokens'].apply(lambda x: [stemmer.stem(i) for i in x])
# cdf['tokenized_tweet'] = cdf['tweet_tokens'].apply(lambda x: ' '.join(x))
# cdf.head()


In [35]:
#Applying the lemmatizer

lemma = WordNetLemmatizer()
test_cdf['tweet'] = test_cdf['tweet'].apply(lambda x: [lemma.lemmatize(i) for i in x])
test_cdf['tweet'] = test_cdf['tweet'].apply(lambda x: ' '.join(x))
test_cdf.head()

,tweet_id,tweet
0,7506,audience q prototyping tool use sketchbooks/sharpie pen photoshop balsamic google doc axsure etc myprototype sxsw
1,7992,sxsw send best photo video link citizen_journalism cnn ireport photography sxsw cyber iphone
2,247,mention pic winning ipad unsix sxsw cc mention mention link cont link
3,7688,google marissa mayer mobile phone cursor physical location new version map fast real life like sxsw
4,3294,sxsw google map even cooler thought


In [36]:
test_df

,tweet_id,tweet
0,7506,"Audience Q: What prototyping tools do you use? Sketchbooks/sharpie pens, photoshop, Balsamic, Google docs, Axsure, etc. #myprototype #sxsw"
1,7992,At SXSW? Send Your Best Photos &amp; Videos to... {link} #citizen_journalism #cnn #ireport #photography #sxsw #Cyber #iPhone
2,247,@mention and here's a pic of you winning your ipad! #unsix #sxsw cc @mention @mention {link} (cont) {link}
3,7688,Google Marissa Mayer: mobile phone as a cursor of physical location - new version of map fast and more real life like #sxsw
4,3294,#SXSW Google maps is even cooler than I thought
...,...,...
1814,1550,@mention @mention @mention Hmmm....how fast can #apple build a new store in time for #sxsw {link}
1815,1933,Samsung Galaxy S II Appears At FCC And Team Android #SXSW Party {link} via @mention
1816,9052,@mention You could buy a new iPad 2 tmrw at the Apple pop-up store at #sxsw: {link}
1817,4219,"Wow very long queue of people at apple pop up store now, some have bought 3 iPads! #sxsw@mention Room#NokiaConnects"


In [15]:
# tfidf = TfidfVectorizer()
# train_vector = tfidf.fit_transform(clean_train_df['tweet_tokens']).toarray()
# test_vector = tfidf.transform(test_cdf['tweet']).toarray()

<IPython.core.display.Javascript object>

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
# cv.set_params(stop_words='english')
# cv.set_params(ngram_range=(1,2))
# cv.set_params(max_df=0.5)
# cv.set_params(min_df=2)
train_vector = cv.fit_transform(clean_train_df['tweet_tokens']).toarray()
test_vector = cv.transform(test_cdf['tweet']).toarray()

In [42]:
from sklearn.metrics import f1_score ,roc_auc_score
from sklearn.naive_bayes import MultinomialNB

In [43]:
nb = MultinomialNB()
nb.fit(train_vector,clean_train_df['sentiment'])
y_pred = nb.predict(test_vector)

In [44]:
pd.Series(y_pred).value_counts()

1    1232
2    555 
0    32  
dtype: int64

In [45]:
result = pd.concat([test_df['tweet_id'],pd.Series(y_pred)],keys=['ID','Y_pred'],axis=1,ignore_index=True,names=['ID','Y_Pred'])
result.columns = ['tweet_id','sentiment']
result1 = result.set_index('tweet_id')

In [46]:
result1.to_csv(r'C:\Data Science\Jupyter Notebook\GitHackathon\result_submissions\naive_bayes_cv_pred.csv')

In [21]:
test_df

,tweet_id,tweet
0,7506,"Audience Q: What prototyping tools do you use? Sketchbooks/sharpie pens, photoshop, Balsamic, Google docs, Axsure, etc. #myprototype #sxsw"
1,7992,At SXSW? Send Your Best Photos &amp; Videos to... {link} #citizen_journalism #cnn #ireport #photography #sxsw #Cyber #iPhone
2,247,@mention and here's a pic of you winning your ipad! #unsix #sxsw cc @mention @mention {link} (cont) {link}
3,7688,Google Marissa Mayer: mobile phone as a cursor of physical location - new version of map fast and more real life like #sxsw
4,3294,#SXSW Google maps is even cooler than I thought
...,...,...
1814,1550,@mention @mention @mention Hmmm....how fast can #apple build a new store in time for #sxsw {link}
1815,1933,Samsung Galaxy S II Appears At FCC And Team Android #SXSW Party {link} via @mention
1816,9052,@mention You could buy a new iPad 2 tmrw at the Apple pop-up store at #sxsw: {link}
1817,4219,"Wow very long queue of people at apple pop up store now, some have bought 3 iPads! #sxsw@mention Room#NokiaConnects"


In [138]:
(test_df['tweet_id'] == result['tweet_id'])

True    1819
Name: tweet_id, dtype: int64